In [54]:
import os
from dotenv import load_dotenv, find_dotenv
path_to_env_file = find_dotenv()
load_dotenv(path_to_env_file)

True

In [2]:
from langchain.schema import HumanMessage
from langchain.chat_models.azure_openai import AzureChatOpenAI

In [3]:
chat_model = AzureChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"),
                             openai_api_base=os.environ.get("OPENAI_API_BASE"),
                             deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT"))


In [21]:
from langchain.tools import tool
from typing import Union
from math import pi

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


In [41]:
from langchain.tools import BaseTool

from datetime import datetime
from math import pi
from typing import Union


  
class CircumferenceTool(BaseTool):
      name = "Circumference calculator"
      description = "use this tool when you need to calculate a circumference using the radius of a circle"

      def _run(self, radius: Union[int, float]):
           
           return float(radius)*2.0*pi

      def _arun(self, radius: int):
           raise NotImplementedError("This tool does not support async")
      
class DateTimeTool(BaseTool):
     name="Get Today's Date and Current Time"
     description="use this tool to get today's date and current time in 12-hour format"

     def _run(self):
          return datetime.today().strftime("%a %d %b %Y, %I:%M%p")
     
     def _arun(self):
          raise NotImplementedError("This tool does not support async")

In [62]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

ggl_search = GoogleSearchAPIWrapper(google_api_key=os.environ.get("GOOGLE_SEARCH_CLIENT_KEY"))

def get_top_results(query):
    return ggl_search.results(query, 10)


ggl_search_tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=get_top_results,
)

ggl_search_tool.run("India")





[{'title': 'India - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/India',
  'snippet': 'India has been a federal republic since 1950, governed through a democratic parliamentary system. It is a pluralistic, multilingual and multi-ethnic society.'},
 {'title': 'National Portal of India',
  'link': 'https://www.india.gov.in/',
  'snippet': 'National Portal of India provides a single-window access to information and services that are electronically delivered from all Government Departments,\xa0...'},
 {'title': 'Incredible India: Official Website for Ministry of Tourism India',
  'link': 'https://www.incredibleindia.org/',
  'snippet': 'India has a myriad of landscapes, great heritage and culture, varied flora and fauna. The country is the most preferred tourist destinations for tourists\xa0...'},
 {'title': 'Government of India, Department of Post',
  'link': 'https://www.indiapost.gov.in/',
  'snippet': 'India Post, Ministry of Communication & Technology.'},
 {'title': 'e-Visa',


In [37]:
from datetime import datetime
print(datetime.today().strftime("%a %d %b %Y, %I:%M%p"))

Sun 04 Feb 2024, 08:20PM


In [42]:
# tools = [multiply, add, exponentiate, CircumferenceTool()]
tools = [CircumferenceTool(), DateTimeTool()]

In [ ]:
# from langchain import hub
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/react")

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(chat_model, tools=tools, prompt=prompt)


In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=10)
agent_executor.invoke(
    {
        "input": "Unfortunately, the Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to its trusty tools and absolutely does NOT try to answer math questions by itself.calculate the circumference of a circle that has a radius of 7.81mm?"
        
    })

---
### Using init_agent method
---

In [9]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
)

In [46]:
from langchain.agents import initialize_agent, AgentType


# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=chat_model,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory,
)

In [ ]:
agent("Tell me today date")



result = agent.invoke("Tell me today date")
result["output"]

In [5]:
from googleapiclient.discovery import build
import os

my_api_key = os.environ.get("GOOGLE_SEARCH_CLIENT_KEY")

my_cse_id = os.environ.get("GOOGLE_CSE_ID")


def google_search(search_term, **kwargs):
    service = build("customsearch", "v1", developerKey=my_api_key)
    res = service.cse().list(q=search_term, cx=my_cse_id, **kwargs).execute()
    return res['items']

results = google_search("latest google news", num=5)

# for result in results:
#     print(result)


output = [{'title':ele['title'],'link':ele['link'],'snippet':ele['snippet']} for ele in results]
print(output)


[{'title': 'Google News', 'link': 'https://news.google.com/topstories?hl=en-US&gl=US&ceid=US%3Aen', 'snippet': 'Comprehensive up-to-date news coverage, aggregated from sources all over the world by Google News.'}, {'title': 'Google News', 'link': 'https://news.google.co.in/', 'snippet': 'Comprehensive, up-to-date news coverage, aggregated from sources all over the world by Google News.'}, {'title': 'Google News', 'link': 'https://news.google.com/?gl=US&hl=en-US&ceid=US:en', 'snippet': 'Comprehensive up-to-date news coverage, aggregated from sources all over the world by Google News ... President Joe Biden wins South Carolina Democratic Primary,\xa0...'}, {'title': 'Google News - Daily Headlines - Apps on Google Play', 'link': 'https://play.google.com/store/apps/details?id=com.google.android.apps.magazines&hl=en_US&gl=US', 'snippet': "Google News is a personalized news aggregator that organizes and highlights what's happening in the world so you can discover more about the stories that\